In this notebook, we will walk through the process of:

1. Initializing the YouTubeDataClient with your API key.
2. Collecting video metadata from one or multiple channels.
3. Retrieving performance statistics for selected videos.
4. Extracting top-level comments left for a set of videos.
5. Extracting all comments left on a set of videos (top-level and replies)

The outputs can be returned as raw dictionaries, Pandas DataFrames, or Polars DataFrames.
You can select the format that best fits into your data pipeline or workflow.

For demonstration purposes, the [CDCodes channel](https://www.youtube.com/@cdcodes) and [homedawg channel](https://www.youtube.com/@homedawg_yt) will be used. These belong to one of the developers of this package. You are free to subsitute these for a channel or channels of your choosing

# Setup - Load API Key and YouTubeDataClient

To generate an API key for your google developer account, please see the [official YouTube API v3 documentation](https://developers.google.com/youtube/v3/getting-started) for more information

In [56]:
# Load environment variables
import os
from yt_stats_wrangler.api.client import YouTubeDataClient


# Get your API key - recommended to store it as an environment variable
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_V3_KEY")

if not YOUTUBE_API_KEY:
    raise ValueError("Missing YOUTUBE_API_V3_KEY. it is recommended that you set it as an environment variable.")

# Create the client (with quoata limit set to the free trial of 10000 units)
client = YouTubeDataClient(api_key=YOUTUBE_API_KEY, max_quota=10000)
print(f"Remaining units in quota: {client.get_remaining_quota()}")

Remaining units in quota: 10000


# Gather Channel IDs using handles

A channel ID will be the core component for gathering data on a channel and its videos. Using an accounts "@" handle, the channel ID can be extracted directly from the API itself.

In [57]:
# Channel ID(s) to test the package
# Feel free to test the package with those of your choosing

# Singular channel case - channel string
yt_handle = "@cdcodes" #https://www.youtube.com/@cdcodes
# Multiple channel - list of channel strings
yt_handles =  ["@cdcodes", "@homedawg_yt"] # https://www.youtube.com/@cdcodes, https://www.youtube.com/@homedawg_yt

# Extract channel IDs from both cases and print the result
TEST_CHANNEL_ID = client.get_channel_id_from_handle(yt_handle)
TEST_CHANNEL_IDS = client.get_channel_ids_from_handles(yt_handles)
print(f"Channel ID for {yt_handle} is {TEST_CHANNEL_ID}")
print(f"Channel IDs for {yt_handles} are {TEST_CHANNEL_IDS}")
print(f"Remaining units in quota: {client.get_remaining_quota()}")

Resolving handle: @cdcodes
Resolving handle: @homedawg_yt
Channel ID for @cdcodes is UCB2mKxxXPK3X8SJkAc-db3A
Channel IDs for ['@cdcodes', '@homedawg_yt'] are ['UCB2mKxxXPK3X8SJkAc-db3A', 'UC0pIFIthf92_8ku2NAYFG_Q']
Remaining units in quota: 9700


# Gather High-Level Channel Summaries

Series of functions that can take a YouTube channel ID and gather the following

+ Name of the Channel
+ Subscribers
+ Number of Uploads
+ Total Channel Views

## Get summaries for a singe channel

### Raw output from YouTube API V3

In [58]:
# Get high-level statistics for a single channel
channel_stats = client.get_channel_statistics(TEST_CHANNEL_ID, key_format="raw", output_format="raw")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
channel_stats

Remaining units in quota: 9699


[{'channelId': 'UCB2mKxxXPK3X8SJkAc-db3A',
  'channelName': 'CDcodes',
  'subscribers': 3620,
  'totalChannelViews': 480975,
  'totalPosts': 18,
  'channelStats_commit_time': '2025-04-03 01:00:34.703539'}]

### Pandas output with modified column names

In [59]:
# Get high-level statistics for a single channel
channel_stats_pandas_df = client.get_channel_statistics(TEST_CHANNEL_ID, key_format="upper", output_format="pandas")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
channel_stats_pandas_df

Remaining units in quota: 9698


,CHANNEL_ID,CHANNEL_NAME,SUBSCRIBERS,TOTAL_CHANNEL_VIEWS,TOTAL_POSTS,CHANNEL_STATS_COMMIT_TIME
0,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,3620,480975,18,2025-04-03 01:00:34.806481


### Polars output with modified column names

In [60]:
# Get high-level statistics for a single channel
channel_stats_polars_df = client.get_channel_statistics(TEST_CHANNEL_ID, key_format="lower", output_format="polars")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
channel_stats_polars_df

Remaining units in quota: 9697


channel_id,channel_name,subscribers,total_channel_views,total_posts,channel_stats_commit_time
str,str,i64,i64,i64,str
"""UCB2mKxxXPK3X8SJkAc-db3A""","""CDcodes""",3620,480975,18,"""2025-04-03 01:00:34.935216"""


## Get summaries for a multiple channels at a time

### Raw output from YouTube API V3

In [61]:
# Get high-level statistics for a single channel
channels_stats = client.get_channel_statistics_for_channels(TEST_CHANNEL_IDS, key_format="raw", output_format="raw")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
channels_stats

Remaining units in quota: 9695


[{'channelId': 'UCB2mKxxXPK3X8SJkAc-db3A',
  'channelName': 'CDcodes',
  'subscribers': 3620,
  'totalChannelViews': 480975,
  'totalPosts': 18,
  'channelStats_commit_time': '2025-04-03 01:00:35.035136'},
 {'channelId': 'UC0pIFIthf92_8ku2NAYFG_Q',
  'channelName': 'homedawg',
  'subscribers': 36,
  'totalChannelViews': 523,
  'totalPosts': 2,
  'channelStats_commit_time': '2025-04-03 01:00:35.135866'}]

In [62]:
channel_stats_pandas_df = client.get_channel_statistics_for_channels(TEST_CHANNEL_IDS, key_format="lower", output_format="pandas")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
channel_stats_pandas_df

Remaining units in quota: 9693


,channel_id,channel_name,subscribers,total_channel_views,total_posts,channel_stats_commit_time
0,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,3620,480975,18,2025-04-03 01:00:35.264598
1,UC0pIFIthf92_8ku2NAYFG_Q,homedawg,36,523,2,2025-04-03 01:00:35.352362


In [63]:
channel_stats_polars_df = client.get_channel_statistics_for_channels(TEST_CHANNEL_IDS, key_format="upper", output_format="polars")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
channel_stats_polars_df

Remaining units in quota: 9691


CHANNEL_ID,CHANNEL_NAME,SUBSCRIBERS,TOTAL_CHANNEL_VIEWS,TOTAL_POSTS,CHANNEL_STATS_COMMIT_TIME
str,str,i64,i64,i64,str
"""UCB2mKxxXPK3X8SJkAc-db3A""","""CDcodes""",3620,480975,18,"""2025-04-03 01:00:35.457903"""
"""UC0pIFIthf92_8ku2NAYFG_Q""","""homedawg""",36,523,2,"""2025-04-03 01:00:35.569604"""


# Gather Video IDs and Metadata

Series of functions that can take a YouTube channel ID or IDs and gather the following

+ Titles, Descriptions, and upload dates for public videos on a YouTube channle or channels
+ Video IDs that can be passed to other methods to collect descriptive data

## Gather video IDs, titles and descriptions from a single channel

### Raw output from YouTube API V3

In [64]:
# Get all public videos from a channel and print out in raw form
videos = client.get_all_video_details_for_channel(TEST_CHANNEL_ID, key_format="raw", output_format='raw')
print(f"Collected {len(videos)} videos.")
print(videos[:1])  # Preview first  result
print(f"Remaining units in quota: {client.get_remaining_quota()}")

Collected 18 videos.
[{'channelId': 'UCB2mKxxXPK3X8SJkAc-db3A', 'videoId': '72Ef65B65JA', 'publishedAt': '2025-02-09T21:01:50Z', 'title': 'Python DeepSeek Tutorial: Installing DeepSeek Locally and Interfacing with Python', 'description': "In this tutorial, we'll be installing DeepSeek onto our local machine, and then interfacing with the LLM through a Python script. I'll also run through a few very basic use cases that can give you an idea of how you might process data and automate tasks through an LLM.\n\nCode and Datasets can be found here: https://github.com/ChristianD37/YoutubeTutorials/tree/master/DeepSeekPython\n\nollama download: https://ollama.com/download\ndeepseek-r1 page: https://ollama.com/library/deepseek-r1\n\n00:00 - Introduction\n00:56 - Installing ollama and DeepSeek locally\n02:05 - Interfacing with DeepSeek through Python\n03:46 - Cleaner Interface Function\n06:54 - Sentiment Analysis on YouTube Comments\n11:24 - Pokemon 3 to 5 sentence Stat Summaries", 'channelTitle

### Pandas output with modified column names

In [65]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_pandas_df = client.get_all_video_details_for_channel(TEST_CHANNEL_ID, key_format="upper", output_format='pandas')
print(f"Collected {videos_pandas_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
videos_pandas_df.head(5)  # Preview first few results

Collected 18 videos.
Remaining units in quota: 9687


,CHANNEL_ID,VIDEO_ID,PUBLISHED_AT,TITLE,DESCRIPTION,CHANNEL_TITLE,VIDEO_DETAILS_COMMIT_TIME
0,UCB2mKxxXPK3X8SJkAc-db3A,72Ef65B65JA,2025-02-09T21:01:50Z,Python DeepSeek Tutorial: Installing DeepSeek ...,"In this tutorial, we'll be installing DeepSeek...",CDcodes,2025-04-03 01:00:36.031221
1,UCB2mKxxXPK3X8SJkAc-db3A,S3dlYi7LPk0,2024-08-18T23:14:38Z,Thank You Pygame,"An ode to pygame, the quirky little python mod...",CDcodes,2025-04-03 01:00:36.031221
2,UCB2mKxxXPK3X8SJkAc-db3A,5OoukJNz3YQ,2023-05-30T00:05:58Z,Beginner Python Project From Scratch: Web Scra...,"In this video, I'll be walking you through a r...",CDcodes,2025-04-03 01:00:36.031221
3,UCB2mKxxXPK3X8SJkAc-db3A,ruISjcokmuk,2022-11-09T01:38:55Z,Beginner Python Project From Scratch: Rock Pap...,Follow along with me on replit: https://replit...,CDcodes,2025-04-03 01:00:36.031221
4,UCB2mKxxXPK3X8SJkAc-db3A,Q6CCdCBVypg,2022-04-05T22:45:48Z,Beginner Python Project From Scratch: Tic Tac Toe,Follow along with me on replit: https://replit...,CDcodes,2025-04-03 01:00:36.031221


### Polars output with modified column names

In [66]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_polars_df = client.get_all_video_details_for_channel(TEST_CHANNEL_ID, key_format="lower", output_format='polars')
print(f"Collected {videos_polars_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
videos_polars_df.head(5)  # Preview first few results

Collected 18 videos.
Remaining units in quota: 9685


channel_id,video_id,published_at,title,description,channel_title,video_details_commit_time
str,str,str,str,str,str,str
"""UCB2mKxxXPK3X8SJkAc-db3A""","""72Ef65B65JA""","""2025-02-09T21:01:50Z""","""Python DeepSeek Tutorial: Inst…","""In this tutorial, we'll be ins…","""CDcodes""","""2025-04-03 01:00:36.281805"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""S3dlYi7LPk0""","""2024-08-18T23:14:38Z""","""Thank You Pygame""","""An ode to pygame, the quirky l…","""CDcodes""","""2025-04-03 01:00:36.281805"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""5OoukJNz3YQ""","""2023-05-30T00:05:58Z""","""Beginner Python Project From S…","""In this video, I'll be walking…","""CDcodes""","""2025-04-03 01:00:36.281805"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""ruISjcokmuk""","""2022-11-09T01:38:55Z""","""Beginner Python Project From S…","""Follow along with me on replit…","""CDcodes""","""2025-04-03 01:00:36.281805"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""Q6CCdCBVypg""","""2022-04-05T22:45:48Z""","""Beginner Python Project From S…","""Follow along with me on replit…","""CDcodes""","""2025-04-03 01:00:36.281805"""


## Gather video IDs, titles and descriptions from multiple channels at a time

### Raw output from YouTube API V3

In [67]:
# Get all public videos from a channel and print out in raw form
videos = client.get_all_video_details_for_channels(TEST_CHANNEL_IDS, key_format="raw", output_format='raw')
print(f"Collected {len(videos)} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
videos[-1]  # Preview tail of the results

Fetching videos for channel: UCB2mKxxXPK3X8SJkAc-db3A
Fetching videos for channel: UC0pIFIthf92_8ku2NAYFG_Q
Collected 20 videos.
Remaining units in quota: 9681


{'channelId': 'UC0pIFIthf92_8ku2NAYFG_Q',
 'videoId': 'vDm8pvsqFY8',
 'publishedAt': '2024-02-14T09:41:13Z',
 'title': 'What Competitive Super Smash Bros Taught Me (About Life and Myself)',
 'description': "Some reflections on personal development that occurred through my journey as an amateur competitor in Super Smash Bros Melee for the Nintendo Gamecube.\n\n\nTimestamps\n00:00:00 - Introduction\n00:01:26 - Lesson 1~ Improvement is a game (and you can level up)\n00:06:10 - Lesson 2~ You're an obstacle in someone else's story\n00:08:45 - Lesson 3~ Losing sucks, detaching is easier\n00:11:04 - Lesson 4~ Win or lose, vulnerability is cool\n00:14:21 - Lesson 5~ Anyone can step up and be a leader\n00:17:08 - Final Remarks\n\nYou should totally play Melee if you don't already, checkout Slippi:\nhttps://slippi.gg/\n\nMusic in order of appearance\nMii Channel (Editing)\nMenu - Super Smash Bros Melee\n7PM - Animal Crossing New Leaf\nIt's Raining Somewhere Else - Toby Fox\nMenu 2 - Super Smash 

### Pandas output with modified column names

In [68]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_pandas_df = client.get_all_video_details_for_channels(TEST_CHANNEL_IDS, key_format="lower", output_format='pandas')
print(f"Collected {videos_pandas_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
videos_pandas_df.tail(5)  # Preview first few results

Fetching videos for channel: UCB2mKxxXPK3X8SJkAc-db3A
Fetching videos for channel: UC0pIFIthf92_8ku2NAYFG_Q
Collected 20 videos.
Remaining units in quota: 9677


,channel_id,video_id,published_at,title,description,channel_title,video_details_commit_time
15,UCB2mKxxXPK3X8SJkAc-db3A,ePiMYe7JpJo,2020-07-27T15:59:04Z,"Pygame Sprite Sheet Tutorial: How to Load, Par...","In this tutorial, we'll be learning all about ...",CDcodes,2025-04-03 01:00:36.971805
16,UCB2mKxxXPK3X8SJkAc-db3A,bmRFi7-gy5Y,2020-07-24T06:33:18Z,Pygame Menu System Tutorial Part 2: Building t...,LINK TO PART 1: https://youtu.be/a5JWrd7Y_14\n...,CDcodes,2025-04-03 01:00:36.971805
17,UCB2mKxxXPK3X8SJkAc-db3A,a5JWrd7Y_14,2020-07-24T06:03:54Z,Pygame Menu System Tutorial Part 1: Game Loops...,LINK TO PART 2: https://www.youtube.com/watch?...,CDcodes,2025-04-03 01:00:36.971805
18,UC0pIFIthf92_8ku2NAYFG_Q,D7YJXgtLqAU,2024-05-05T23:21:21Z,"Nostalgia, Change and Pokemon",Examining my relationship with the greatest ma...,homedawg,2025-04-03 01:00:37.177255
19,UC0pIFIthf92_8ku2NAYFG_Q,vDm8pvsqFY8,2024-02-14T09:41:13Z,What Competitive Super Smash Bros Taught Me (A...,Some reflections on personal development that ...,homedawg,2025-04-03 01:00:37.177255


### Polars output with modified column names

In [69]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_polars_df = client.get_all_video_details_for_channels(TEST_CHANNEL_IDS, key_format="upper", output_format='polars')
print(f"Collected {videos_polars_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
videos_polars_df.tail(5)  # Preview last few results

Fetching videos for channel: UCB2mKxxXPK3X8SJkAc-db3A
Fetching videos for channel: UC0pIFIthf92_8ku2NAYFG_Q
Collected 20 videos.
Remaining units in quota: 9673


CHANNEL_ID,VIDEO_ID,PUBLISHED_AT,TITLE,DESCRIPTION,CHANNEL_TITLE,VIDEO_DETAILS_COMMIT_TIME
str,str,str,str,str,str,str
"""UCB2mKxxXPK3X8SJkAc-db3A""","""ePiMYe7JpJo""","""2020-07-27T15:59:04Z""","""Pygame Sprite Sheet Tutorial: …","""In this tutorial, we'll be lea…","""CDcodes""","""2025-04-03 01:00:37.397189"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""bmRFi7-gy5Y""","""2020-07-24T06:33:18Z""","""Pygame Menu System Tutorial Pa…","""LINK TO PART 1: https://youtu.…","""CDcodes""","""2025-04-03 01:00:37.397189"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""a5JWrd7Y_14""","""2020-07-24T06:03:54Z""","""Pygame Menu System Tutorial Pa…","""LINK TO PART 2: https://www.yo…","""CDcodes""","""2025-04-03 01:00:37.397189"""
"""UC0pIFIthf92_8ku2NAYFG_Q""","""D7YJXgtLqAU""","""2024-05-05T23:21:21Z""","""Nostalgia, Change and Pokemon""","""Examining my relationship with…","""homedawg""","""2025-04-03 01:00:37.612613"""
"""UC0pIFIthf92_8ku2NAYFG_Q""","""vDm8pvsqFY8""","""2024-02-14T09:41:13Z""","""What Competitive Super Smash B…","""Some reflections on personal d…","""homedawg""","""2025-04-03 01:00:37.612613"""


# Get Video Statistics

Once you have a list of video IDs, you can retrieve high-level public statistics
for each video, such as 
+ View counts
+ Like counts
+ Comment counts
+ Tags

This is useful for performance analytics and benchmarking.

### Raw output from YouTube API V3

In [70]:
# Let's grab the first 5 video IDs from the previous data
video_ids = [video["videoId"] for video in videos[:5]]

# Get statistics for the videos
video_stats = client.get_video_stats(video_ids, key_format="raw", output_format="raw")
print(f"Retrieved statistics for {len(video_stats)} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
video_stats[0]

Retrieved statistics for 5 videos.
Remaining units in quota: 9672


{'videoId': '72Ef65B65JA',
 'title': 'Python DeepSeek Tutorial: Installing DeepSeek Locally and Interfacing with Python',
 'description': "In this tutorial, we'll be installing DeepSeek onto our local machine, and then interfacing with the LLM through a Python script. I'll also run through a few very basic use cases that can give you an idea of how you might process data and automate tasks through an LLM.\n\nCode and Datasets can be found here: https://github.com/ChristianD37/YoutubeTutorials/tree/master/DeepSeekPython\n\nollama download: https://ollama.com/download\ndeepseek-r1 page: https://ollama.com/library/deepseek-r1\n\n00:00 - Introduction\n00:56 - Installing ollama and DeepSeek locally\n02:05 - Interfacing with DeepSeek through Python\n03:46 - Cleaner Interface Function\n06:54 - Sentiment Analysis on YouTube Comments\n11:24 - Pokemon 3 to 5 sentence Stat Summaries",
 'publishedAt': '2025-02-09T21:01:50Z',
 'channelId': 'UCB2mKxxXPK3X8SJkAc-db3A',
 'channelTitle': 'CDcodes',
 't

### Pandas output with modified column names

In [71]:
# Let's grab the first 5 video IDs from the pandas dataframe
# Get statistics for the videos
video_stats_pandas_df = client.get_video_stats(videos_pandas_df['video_id'][0:5], key_format="lower", output_format="pandas")
print(f"Retrieved statistics for {video_stats_pandas_df .shape[0]} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
video_stats_pandas_df.head()

Retrieved statistics for 5 videos.
Remaining units in quota: 9671


,video_id,title,description,published_at,channel_id,channel_title,tags,category_id,view_count,like_count,comment_count,duration_seconds,definition,is_short,video_stats_commit_time
0,72Ef65B65JA,Python DeepSeek Tutorial: Installing DeepSeek ...,"In this tutorial, we'll be installing DeepSeek...",2025-02-09T21:01:50Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,4720,95,18,925,hd,False,2025-04-03 01:00:37.863502
1,S3dlYi7LPk0,Thank You Pygame,"An ode to pygame, the quirky little python mod...",2024-08-18T23:14:38Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,8753,429,37,400,hd,False,2025-04-03 01:00:37.863502
2,5OoukJNz3YQ,Beginner Python Project From Scratch: Web Scra...,"In this video, I'll be walking you through a r...",2023-05-30T00:05:58Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,1552,68,11,2737,hd,False,2025-04-03 01:00:37.863502
3,ruISjcokmuk,Beginner Python Project From Scratch: Rock Pap...,Follow along with me on replit: https://replit...,2022-11-09T01:38:55Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,3331,55,9,960,hd,False,2025-04-03 01:00:37.863502
4,Q6CCdCBVypg,Beginner Python Project From Scratch: Tic Tac Toe,Follow along with me on replit: https://replit...,2022-04-05T22:45:48Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,79190,809,65,1014,hd,False,2025-04-03 01:00:37.863502


### Polars output with modified column names

In [72]:
# Let's grab the first 5 video IDs from the pandas dataframe
# Get statistics for the videos
video_stats_polars_df = client.get_video_stats(videos_polars_df['VIDEO_ID'][0:5], key_format="upper", output_format="pandas")
print(f"Retrieved statistics for {video_stats_polars_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
video_stats_polars_df.head()

Retrieved statistics for 5 videos.
Remaining units in quota: 9670


,VIDEO_ID,TITLE,DESCRIPTION,PUBLISHED_AT,CHANNEL_ID,CHANNEL_TITLE,TAGS,CATEGORY_ID,VIEW_COUNT,LIKE_COUNT,COMMENT_COUNT,DURATION_SECONDS,DEFINITION,IS_SHORT,VIDEO_STATS_COMMIT_TIME
0,72Ef65B65JA,Python DeepSeek Tutorial: Installing DeepSeek ...,"In this tutorial, we'll be installing DeepSeek...",2025-02-09T21:01:50Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,4720,95,18,925,hd,False,2025-04-03 01:00:37.990106
1,S3dlYi7LPk0,Thank You Pygame,"An ode to pygame, the quirky little python mod...",2024-08-18T23:14:38Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,8753,429,37,400,hd,False,2025-04-03 01:00:37.990106
2,5OoukJNz3YQ,Beginner Python Project From Scratch: Web Scra...,"In this video, I'll be walking you through a r...",2023-05-30T00:05:58Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,1552,68,11,2737,hd,False,2025-04-03 01:00:37.990106
3,ruISjcokmuk,Beginner Python Project From Scratch: Rock Pap...,Follow along with me on replit: https://replit...,2022-11-09T01:38:55Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,3331,55,9,960,hd,False,2025-04-03 01:00:37.990106
4,Q6CCdCBVypg,Beginner Python Project From Scratch: Tic Tac Toe,Follow along with me on replit: https://replit...,2022-04-05T22:45:48Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,79190,809,65,1014,hd,False,2025-04-03 01:00:37.990106


# Get Video Comments

Once you have a list of video IDs, you can the following data regarding each of the comments
+ Comment Text
+ Like counts
+ Reply counts
+ Publication Date

This is useful for performance analytics and sentiment analysis

## Gather top level comments for videos

These methods will not include nested comments. In other words, it will gather comments left on the video, but not comments replying to other comments. See the next section to gather all comments.

### Raw output from YouTube API V3

In [73]:
comments = client.get_top_level_comments_for_video_ids(video_ids, key_format="raw", output_format="raw")
print(f"Collected {len(comments)} comments across {len(video_ids)} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
comments[0:2]

Fetching comments for video ID: 72Ef65B65JA
Fetching comments for video ID: S3dlYi7LPk0
Fetching comments for video ID: 5OoukJNz3YQ
Fetching comments for video ID: ruISjcokmuk
Fetching comments for video ID: Q6CCdCBVypg
Collected 74 comments across 5 videos.
Remaining units in quota: 9665


[{'videoId': '72Ef65B65JA',
  'commentId': 'UgwerLKK8iN9HRRWhQB4AaABAg',
  'author': '@PARK60264',
  'text': 'Is this local? lama is not local is it?',
  'publishedAt': '2025-03-29T00:12:15Z',
  'likeCount': 0,
  'replyCount': 1,
  'videoTopLevelComments_commit_time': '2025-04-03 01:00:38.122877'},
 {'videoId': '72Ef65B65JA',
  'commentId': 'Ugwt5Tcy5DR98EpXUfF4AaABAg',
  'author': '@xiaoyanzheng6435',
  'text': 'hi thank you for sharing this very helpful! for me it runs a bit slow but still works in the command prompt but when i switched to Jupyter Notebook, it slowed dramatically, a simple question takes 10 minutes for the answer to be produced. Any advice on improving speed? thanks!',
  'publishedAt': '2025-02-27T15:55:04Z',
  'likeCount': 0,
  'replyCount': 1,
  'videoTopLevelComments_commit_time': '2025-04-03 01:00:38.122877'}]

### Pandas output with modified column names

In [74]:
# Get top-level comments from the same 5 videos
comments_pandas_df = client.get_top_level_comments_for_video_ids(videos_pandas_df['video_id'][0:5], key_format="lower", output_format="pandas")
print(f"Collected {comments_pandas_df.shape[0]} comments across {len(videos_pandas_df['video_id'][0:5].unique())} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
comments_pandas_df.head(5)

Fetching comments for video ID: 72Ef65B65JA
Fetching comments for video ID: S3dlYi7LPk0
Fetching comments for video ID: 5OoukJNz3YQ
Fetching comments for video ID: ruISjcokmuk
Fetching comments for video ID: Q6CCdCBVypg
Collected 74 comments across 5 videos.
Remaining units in quota: 9660


,video_id,comment_id,author,text,published_at,like_count,reply_count,video_top_level_comments_commit_time
0,72Ef65B65JA,UgwerLKK8iN9HRRWhQB4AaABAg,@PARK60264,Is this local? lama is not local is it?,2025-03-29T00:12:15Z,0,1,2025-04-03 01:00:38.759039
1,72Ef65B65JA,Ugwt5Tcy5DR98EpXUfF4AaABAg,@xiaoyanzheng6435,hi thank you for sharing this very helpful! fo...,2025-02-27T15:55:04Z,0,1,2025-04-03 01:00:38.759039
2,72Ef65B65JA,UgzgAHrcz8j51SwKlYN4AaABAg,@googleadmin4749,"Halfpoker is misleading, it uses the Lama crut...",2025-02-21T08:48:10Z,0,2,2025-04-03 01:00:38.759039
3,72Ef65B65JA,Ugysv5Erz5c84y_Fsrd4AaABAg,@carloscastc,"Dude this is an excellent tutorial, hope you u...",2025-02-20T21:44:55Z,0,1,2025-04-03 01:00:38.759039
4,72Ef65B65JA,Ugym5WKbqvgNqQdqTgh4AaABAg,@appsenence9244,Idk why I'm getting this error while trying to...,2025-02-19T14:56:06Z,0,4,2025-04-03 01:00:38.759039


### Polars output with modified column names

In [75]:
# Get top-level comments from the same 5 videos
comments_polars_df = client.get_top_level_comments_for_video_ids(videos_polars_df['VIDEO_ID'][0:5], key_format="upper", output_format="polars")
print(f"Collected {comments_polars_df.shape[0]} comments across {len(comments_polars_df['VIDEO_ID'][0:5].unique())} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
comments_polars_df.tail(5)

Fetching comments for video ID: 72Ef65B65JA
Fetching comments for video ID: S3dlYi7LPk0
Fetching comments for video ID: 5OoukJNz3YQ
Fetching comments for video ID: ruISjcokmuk
Fetching comments for video ID: Q6CCdCBVypg
Collected 74 comments across 1 videos.
Remaining units in quota: 9655


VIDEO_ID,COMMENT_ID,AUTHOR,TEXT,PUBLISHED_AT,LIKE_COUNT,REPLY_COUNT,VIDEO_TOP_LEVEL_COMMENTS_COMMIT_TIME
str,str,str,str,str,i64,i64,str
"""Q6CCdCBVypg""","""UgxhHvu2akHf8JBjU6h4AaABAg""","""@Poxlol""","""Underrated""","""2022-04-23T03:21:11Z""",2,1,"""2025-04-03 01:00:39.872523"""
"""Q6CCdCBVypg""","""UgxUlgYS3l9AXkQnuWl4AaABAg""","""@phoebely2900""","""Fantastic work!""","""2022-04-06T04:44:48Z""",1,1,"""2025-04-03 01:00:39.872523"""
"""Q6CCdCBVypg""","""UgyukXWFPcH1fo7Imv54AaABAg""","""@chosenone2435""","""Great to see you back! You are…","""2022-04-06T01:25:19Z""",4,1,"""2025-04-03 01:00:39.872523"""
"""Q6CCdCBVypg""","""UgxfNe6LcnteSh0uD3h4AaABAg""","""@joey546""","""Good to see you back! You stil…","""2022-04-05T23:35:12Z""",9,2,"""2025-04-03 01:00:39.872523"""
"""Q6CCdCBVypg""","""UgybfdmW11TCtS82wjh4AaABAg""","""@MyFishWillKillU""","""Wow so cool""","""2022-04-05T22:53:18Z""",4,0,"""2025-04-03 01:00:39.872523"""


## Get all comments and comment replies for videos

These methods will include nested comments. In other words, it will gather comments left on the video, along with comments replying to other comments. It will return comments in the long format (i.e each comment and reply gets it own row or entry), but the `parentId` can be used to keep track of the nested structure of each comment.

In [76]:
# Gather all comments for a sample of 5 videos. See comment count in this method compared to the ones above
all_comments = client.get_all_comments_for_video_ids(video_ids, key_format="raw", output_format="raw")
print(f"Collected {len(all_comments)} comments across {len(video_ids)} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
all_comments[0:2]

Fetching all comments for video ID: 72Ef65B65JA
Fetching all comments for video ID: S3dlYi7LPk0
Fetching all comments for video ID: 5OoukJNz3YQ
Fetching all comments for video ID: ruISjcokmuk
Fetching all comments for video ID: Q6CCdCBVypg
Collected 140 comments across 5 videos.
Remaining units in quota: 9600


[{'commentId': 'UgwerLKK8iN9HRRWhQB4AaABAg',
  'videoId': '72Ef65B65JA',
  'author': '@PARK60264',
  'text': 'Is this local? lama is not local is it?',
  'publishedAt': '2025-03-29T00:12:15Z',
  'likeCount': 0,
  'replyCount': 1,
  'parentId': None,
  'videoAllComments_commit_time': '2025-04-03 01:00:40.017127'},
 {'commentId': 'UgwerLKK8iN9HRRWhQB4AaABAg.AGEJqym_U1oAGJ-AbRIA54',
  'parentId': 'UgwerLKK8iN9HRRWhQB4AaABAg',
  'author': '@CDcodes',
  'text': 'Yes if you follow the install instructions it will run deepseek off of your local computer',
  'publishedAt': '2025-03-30T19:47:46Z',
  'likeCount': 0,
  'videoId': None,
  'videoAllComments_commit_time': '2025-04-03 01:00:40.110876'}]

In [77]:
all_comments_pandas_df = client.get_all_comments_for_video_ids(videos_pandas_df['video_id'][0:5], key_format="lower", output_format="pandas")
print(f"Collected {all_comments_pandas_df.shape[0]} comments across {len(videos_pandas_df['video_id'][0:5].unique())} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
all_comments_pandas_df.head(10)

Fetching all comments for video ID: 72Ef65B65JA
Fetching all comments for video ID: S3dlYi7LPk0
Fetching all comments for video ID: 5OoukJNz3YQ
Fetching all comments for video ID: ruISjcokmuk
Fetching all comments for video ID: Q6CCdCBVypg
Collected 140 comments across 5 videos.
Remaining units in quota: 9545


,comment_id,video_id,author,text,published_at,like_count,reply_count,parent_id,video_all_comments_commit_time
0,UgwerLKK8iN9HRRWhQB4AaABAg,72Ef65B65JA,@PARK60264,Is this local? lama is not local is it?,2025-03-29T00:12:15Z,0,1.0,None,2025-04-03 01:00:46.216355
1,UgwerLKK8iN9HRRWhQB4AaABAg.AGEJqym_U1oAGJ-AbRIA54,None,@CDcodes,Yes if you follow the install instructions it ...,2025-03-30T19:47:46Z,0,NaN,UgwerLKK8iN9HRRWhQB4AaABAg,2025-04-03 01:00:46.308110
2,Ugwt5Tcy5DR98EpXUfF4AaABAg,72Ef65B65JA,@xiaoyanzheng6435,hi thank you for sharing this very helpful! fo...,2025-02-27T15:55:04Z,0,1.0,None,2025-04-03 01:00:46.309107
3,Ugwt5Tcy5DR98EpXUfF4AaABAg.AF2kuCt8XAHAFDjze7l89D,None,@CDcodes,Thanks for watching! Not sure why you're seein...,2025-03-03T22:18:44Z,0,NaN,Ugwt5Tcy5DR98EpXUfF4AaABAg,2025-04-03 01:00:46.427789
4,UgzgAHrcz8j51SwKlYN4AaABAg,72Ef65B65JA,@googleadmin4749,"Halfpoker is misleading, it uses the Lama crut...",2025-02-21T08:48:10Z,0,2.0,None,2025-04-03 01:00:46.428786
5,UgzgAHrcz8j51SwKlYN4AaABAg.AEnYHCE2OBBAFDkIjYjRNe,None,@CDcodes,Sorry to hear you didn't enjoy the video. If t...,2025-03-03T22:21:28Z,0,NaN,UgzgAHrcz8j51SwKlYN4AaABAg,2025-04-03 01:00:46.537496
6,UgzgAHrcz8j51SwKlYN4AaABAg.AEnYHCE2OBBAGEJyK8fAmJ,None,@PARK60264,@@CDcodes Sounds like AI generated response lol.,2025-03-29T00:13:15Z,0,NaN,UgzgAHrcz8j51SwKlYN4AaABAg,2025-04-03 01:00:46.537496
7,Ugysv5Erz5c84y_Fsrd4AaABAg,72Ef65B65JA,@carloscastc,"Dude this is an excellent tutorial, hope you u...",2025-02-20T21:44:55Z,0,1.0,None,2025-04-03 01:00:46.538493
8,Ugysv5Erz5c84y_Fsrd4AaABAg.AEmMNPbHdI7AFDk8QJqy3Y,None,@CDcodes,Thanks for watching! I haven't played around w...,2025-03-03T22:20:04Z,0,NaN,Ugysv5Erz5c84y_Fsrd4AaABAg,2025-04-03 01:00:46.653186
9,Ugym5WKbqvgNqQdqTgh4AaABAg,72Ef65B65JA,@appsenence9244,Idk why I'm getting this error while trying to...,2025-02-19T14:56:06Z,0,4.0,None,2025-04-03 01:00:46.653186


In [78]:
all_comments_polars_df = client.get_all_comments_for_video_ids(comments_polars_df['VIDEO_ID'][0:5], key_format="upper", output_format="polars")
print(f"Collected {all_comments_polars_df.shape[0]} comments across {len(comments_polars_df['VIDEO_ID'][0:5].unique())} videos.")
print(f"Remaining units in quota: {client.get_remaining_quota()}")
all_comments_polars_df.tail(10)

Fetching all comments for video ID: 72Ef65B65JA
Fetching all comments for video ID: 72Ef65B65JA
Fetching all comments for video ID: 72Ef65B65JA
Fetching all comments for video ID: 72Ef65B65JA
Fetching all comments for video ID: 72Ef65B65JA
Collected 90 comments across 1 videos.
Remaining units in quota: 9505


COMMENT_ID,VIDEO_ID,AUTHOR,TEXT,PUBLISHED_AT,LIKE_COUNT,REPLY_COUNT,PARENT_ID,VIDEO_ALL_COMMENTS_COMMIT_TIME
str,str,str,str,str,i64,i64,str,str
"""Ugysv5Erz5c84y_Fsrd4AaABAg.AEm…",null,"""@CDcodes""","""Thanks for watching! I haven't…","""2025-03-03T22:20:04Z""",0,null,"""Ugysv5Erz5c84y_Fsrd4AaABAg""","""2025-04-03 01:00:56.083239"""
"""Ugym5WKbqvgNqQdqTgh4AaABAg""","""72Ef65B65JA""","""@appsenence9244""","""Idk why I'm getting this error…","""2025-02-19T14:56:06Z""",0,4,null,"""2025-04-03 01:00:56.084237"""
"""Ugym5WKbqvgNqQdqTgh4AaABAg.AEj…",null,"""@appsenence9244""","""Never mind... I had to pull th…","""2025-02-19T15:34:29Z""",0,null,"""Ugym5WKbqvgNqQdqTgh4AaABAg""","""2025-04-03 01:00:56.184967"""
"""Ugym5WKbqvgNqQdqTgh4AaABAg.AEj…",null,"""@korsik4559""","""@@appsenence9244 change 'model…","""2025-02-20T22:34:08Z""",1,null,"""Ugym5WKbqvgNqQdqTgh4AaABAg""","""2025-04-03 01:00:56.184967"""
"""Ugym5WKbqvgNqQdqTgh4AaABAg.AEj…",null,"""@CDcodes""","""thanks for helping out here : …","""2025-03-03T22:40:44Z""",0,null,"""Ugym5WKbqvgNqQdqTgh4AaABAg""","""2025-04-03 01:00:56.184967"""
"""Ugym5WKbqvgNqQdqTgh4AaABAg.AEj…",null,"""@appsenence9244""","""@@korsik4559 i used a differen…","""2025-03-03T22:52:30Z""",0,null,"""Ugym5WKbqvgNqQdqTgh4AaABAg""","""2025-04-03 01:00:56.184967"""
"""Ugy2GPnoS69usGOCMwp4AaABAg""","""72Ef65B65JA""","""@alikayhanatay9080""","""great, easy to follow tutorial…","""2025-02-16T18:31:55Z""",1,1,null,"""2025-04-03 01:00:56.184967"""
"""Ugy2GPnoS69usGOCMwp4AaABAg.AEb…",null,"""@CDcodes""","""Thank you! I'm planning on upl…","""2025-02-16T18:55:35Z""",1,null,"""Ugy2GPnoS69usGOCMwp4AaABAg""","""2025-04-03 01:00:56.281708"""
"""UgxpBojUlkiwpvUlzAt4AaABAg""","""72Ef65B65JA""","""@MyFishWillKillU""","""First 😂""","""2025-02-09T21:05:08Z""",0,1,null,"""2025-04-03 01:00:56.282706"""
